In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import os 
import seaborn as sns 
import matplotlib.image as mpimg

In [ ]:
train_input = "../input/digit-recognizer/train.csv"
test_input = "../input/digit-recognizer/test.csv"

train_dataset = pd.read_csv(train_input)
test_dataset = pd.read_csv(test_input)

In [ ]:
train_dataset.head()

In [ ]:
test_dataset.head()

In [ ]:
plt.bar(range(10), train_dataset.label.value_counts(), color="#FF6F00", tick_label=range(10))
plt.xlabel("Numbers")
plt.ylabel("Counts")
plt.show()

In [ ]:
train_labels = tf.keras.utils.to_categorical(train_dataset.pop("label"))

In [ ]:
train_dataset = np.array(train_dataset.values.reshape(-1, 28, 28, 1))
test_dataset = np.array(test_dataset.values.reshape(-1, 28, 28, 1))

In [ ]:
train_dataset = train_dataset/255.0
test_dataset = test_dataset/255.0

In [ ]:
checkpoint_path = "logs/checkpoints/"

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation=tf.nn.relu, padding="SAME"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu, padding="SAME"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (3, 3), activation=tf.nn.relu, padding="SAME"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation=tf.nn.relu, padding="SAME"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

epochs=50

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                        monitor="accuracy",
                                        save_best_only=True,
                                        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=15)
]

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_dataset, train_labels, epochs=epochs, callbacks=callbacks, batch_size=64)

In [ ]:
def plot_metric(metric):
    plt.plot(range(epochs), history.history[metric], color="#FF6F00")
    plt.title(metric)
    plt.show()
    
plot_metric("accuracy")
plot_metric("loss")

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
labels = [np.argmax(predict) for predict in model.predict(test_dataset)]
df = pd.DataFrame({
    "ImageId": list(range(1, len(test_dataset)+1)),
    "Label": labels,
})

In [ ]:
df.to_csv("submission.csv", index=False)

In [ ]:
model.save("model.h5")